# Example State Machine Usage



In [1]:
from state_machine import StateMachine, State, Model, Condition, AlwaysTrue, AlwaysFalse, Event

Make some conditions. Not that PhoneRings listens for 'phone', and CustomerWants listerns for 'customer'.

In [2]:
class PhoneRings(Condition):
    
    listens_for='phone'
    
    def __init__(self):
        self.name='Phone Rings'
        
    def is_triggered(self, event):
        return event.payload=='ring ring'
    
PHONE_RINGS = PhoneRings()
assert PHONE_RINGS.is_triggered(Event(payload='ring ring'))==True
assert PHONE_RINGS.is_triggered(Event(payload='silence'))==False

In [3]:
class CustomerWants(Condition):
    listens_for='customer'
    def __init__(self, wants):
        self.name='Wants %s'%wants
        self.wants=wants
        
    def is_triggered(self, event):
        return self.wants==event.payload['wants']

CUSTOMER_WANTS_PIZZA   = CustomerWants('pizza')
CUSTOMER_WANTS_LASAGNE = CustomerWants('lasagne')

assert CUSTOMER_WANTS_PIZZA.is_triggered(Event('customer',{'wants':'lasagne'}))==False
assert CUSTOMER_WANTS_PIZZA.is_triggered(Event('customer',{'wants':'pizza'}))==True

In [15]:
class EnterPizzaOrder(State):
    def on_start(self, event):
        print('Entering Pizza Order')
        
class EnterLasagneOrder(State):
    def on_start(self, event):
        print('Entering Lasagne Order')

In [16]:
receptionist = StateMachine( 'Receptionist' )

wait_for_phone    = State('Wait For Phone')
take_order        = State('Take Order')

enter_lasagne_order = EnterLasagneOrder( 'Enter Lasagne Order' )
enter_pizza_order   = EnterPizzaOrder('Enter Pizza Order' )

wait_for_phone.add_transition_to(take_order,PHONE_RINGS)
take_order.add_transition_to( enter_lasagne_order, CUSTOMER_WANTS_LASAGNE )
take_order.add_transition_to( enter_pizza_order, CUSTOMER_WANTS_PIZZA )

### Receptionist waits, but only receives silence

Broadcast silence on the phone.

In [17]:
evt = Event('phone','silence')
receptionist.set_state(wait_for_phone)
receptionist.handle_event(evt)
assert wait_for_phone==receptionist.current_state

### Phone rings, so receptionist takes order

Broadcast a ringing phone.

In [18]:
evt = Event('phone','ring ring')
receptionist.set_state(wait_for_phone)
receptionist.handle_event(evt)
assert take_order==receptionist.current_state, receptionist.current_state

INFO:default:Starting state Take Order
INFO:default:Take Order started successfully with result: None


### Customer Wants Pizza

Take an order from a customer who wants pizza.

In [19]:
evt = Event('customer',{'wants':'pizza'})
receptionist.set_state( take_order )
receptionist.handle_event(evt)
assert enter_pizza_order==receptionist.current_state

INFO:default:Starting state Enter Pizza Order
INFO:default:Enter Pizza Order started successfully with result: None


Entering Pizza Order


### Customer Wants Lasagne

Take an order from a customer who wants pizza.

In [24]:
evt = Event('customer',{'wants':'lasagne'})
receptionist.set_state( take_order )
receptionist.handle_event(evt)
assert enter_lasagne_order==receptionist.current_state

INFO:default:Starting state Enter Lasagne Order
INFO:default:Enter Lasagne Order started successfully with result: None


Entering Lasagne Order


### The Full Pizza Sequence

Test a ringing phone, followed by a customer who wants pizza.

In [29]:
receptionist.set_state( wait_for_phone )
receptionist.handle_event(Event('phone', 'silence'))
receptionist.handle_event(Event('phone', 'ring ring'))
receptionist.handle_event(Event('customer', {'wants':'pizza'}))
assert enter_pizza_order==receptionist.current_state

INFO:default:Starting state Take Order
INFO:default:Take Order started successfully with result: None
INFO:default:Starting state Enter Pizza Order
INFO:default:Enter Pizza Order started successfully with result: None


Entering Pizza Order


### The Full Pizza Sequence

Test a ringing phone, followed by a customer who wants lasagne.

In [30]:
receptionist.set_state( wait_for_phone )
receptionist.handle_event(Event('phone', 'silence'))
receptionist.handle_event(Event('phone', 'ring ring'))
receptionist.handle_event(Event('customer', {'wants':'lasagne'}))
assert enter_lasagne_order==receptionist.current_state

INFO:default:Starting state Take Order
INFO:default:Take Order started successfully with result: None
INFO:default:Starting state Enter Lasagne Order
INFO:default:Enter Lasagne Order started successfully with result: None


Entering Lasagne Order
